# Projeto 3 - MC886

## Alunos: 
&emsp;&emsp;&emsp;&emsp;Felipe Escórcio de Sousa - RA:171043\
&emsp;&emsp;&emsp;&emsp;Miguel Augusto S Guida&nbsp;&nbsp;&nbsp;- RA:174847

## Objetivo
&emsp;Este projeto tem como objetivo aplicar os conceitos de Algoritmos Evolutivos e Métodos de Aprendizagem por Reforço para treinar um modelo capaz de jogar Pac-Man. O trabalho foi então dividido em duas partes:

<li>
    <ul>Parte 1: Modelo Evolutivo</ul>
    <ul>Parte 2: Modelo de Aprendizagem por Reforço</ul>
</li>

## Jogo do Pacman
&emsp;PACMAN MELHOR JOGO PQP 
&emsp;&emsp;Falar de estado e acao. Nao deixamos a acao STOP como uma opcao disponivel. Como funciona o jogo, quais variaveis de estado utilizamos. 

## Parte 1 - Modelo Evolutivo
&emsp;blablabla


In [1]:
from search import *
from game import *
from util import *
from pacman import *
import util, layout
import sys, types, time, random, os
import numpy as np
import math
import matplotlib.pyplot as plt
from random import random, randint, seed, choice

## Parte 2 - Modelo de Aprendizagem por Reforço
&emsp;&emsp;Para nosso modelo de aprendizagem por reforço, levamos em consideração que o agente do jogo precisava estar a par do ambiente para que ele tome uma decisão. Assim, utilizamos as mesmas variáveis de ambiente do algoritmo evolutivo para compor nosso algoritmo.\
&emsp;&emsp;Nossa ideia inicial seria implementar um algoritmo de aprendizagem por reforço que permitisse ao agente tomar decisões baseadas nas observações do estado atual e nas suposições do próximo estado, sem depender de uma tabela de valores para rankear os estados, já que o jogo é composto por inimigos que se movem aleatoriamente.\
&emsp;&emsp;Com isso em mente, adotamos o algoritmo Approximate Q-Learning para resolver o problema, pois ele é policy free e model free, e a atualização dos pesos de Q são obtidos a cada ação do agente.
### MDP (Markov Decision Process)
&emsp;Definimos nosso estado como sendo o estado atual e o proximo estado, dado cada acao disponivel no estado atual. ?



### Features
&emsp;Escolhemos as features para nosso modelo de acordo com o que achamos mais importante para a sobrevivência e sucesso do agente durante o jogo. As features relacionadas a distância foram calculadas utilizando o algoritmo BFS (breadth first search), assim conseguimos calcular a distância entre o pacman e o objeto de interesse (fantasmas, comida ou pilula). As features escolhidas são:
- **futureScore**: composto pela diferença entre score futuro e score atual, somado com as variáveis minGhostDist$*200$ (caso minGhostDist = 1) e minDistToFood$*200$ (caso minGhostDist < 1)
- **minGhostDist**: distância para o fantasma perigoso mais próximo ou zero (caso edibleGhost = 1)
- **minDistToFood**: distância para a comida mais próxima 
- **distToPill**: distância para a pilula mais próxima.
- **edibleGhost**: distância para o fantasma comível mais próximo.
&emsp;Para nosso modelo aprender com mais facilidade, normalizamos o valor absoluto das features de distancia entre 0 e 1, computando o valor da feature atraves da funcao $f(x)=1/x^2$. Para o tratamento da feature *futureScore*, dividimos o valor calculado por 500, pois eh o valor maximo absoluto do score de uma acao. Essas configuracoes foram implementadas na intenção de ajudar o modelo a convergir e encontrar valores em uma mesma escala para os pesos.

### Rewards
&emsp;&emsp;As Rewards são calculadas de acordo com a feature *futureScore*, recebendo o valor da feature sem a multiplicação com o peso. Como descrito, uma lógica de reforço positivo e negativo é realizada no cálculo da feature. Isso contribuiu para que o agente aprendesse a a fugir dos fantasmas e ao mesmo tempo fosse em busca das comidas.

In [2]:

"Calcula a distancia minima do agente para as posicoes na lista positionsInput. "
def minDistBFS(agentPos, positionsInput, wallGrid):
    positions = []
    for pos in positionsInput:
        positions.append((math.floor(pos[0]),math.floor(pos[1])))
    # Toma paredes como celulas ja visitadas
    visited = wallGrid.copy()
    queue = []
    queue.append((agentPos,1))
    visited[agentPos[0]][agentPos[1]] = True
    if len(positions) > 0:
        if(agentPos in positions):
            return 1
        while queue:
            el = queue.pop(0)
            h = el[1]
            s = el[0]

            north = (s[0],s[1]+1)
            south = (s[0],s[1]-1)
            west = (s[0]-1,s[1])
            east = (s[0]+1,s[1])

            for i in [north, south, west, east]:    
                if visited[i[0]][i[1]] == False:
                    queue.append((i,h+1))
                    visited[i[0]][i[1]] = True

                    if(i in positions):    
                        return h
                
    return 0

"Calcula a distancia minima do agente para a comida mais proxima. "
def findFoodMinDistBFS(agentPos, foodGrid, wallGrid):
    # Toma paredes como celulas ja visitadas
    visited = wallGrid.copy()
    queue = []
    queue.append((agentPos,1))
    visited[agentPos[0]][agentPos[1]] = True
    if(foodGrid[agentPos[0]][agentPos[1]]):  
        return 1
    while queue:
        el = queue.pop(0)
        h = el[1]
        s = el[0]

        north = (s[0],s[1]+1)
        south = (s[0],s[1]-1)
        west = (s[0]-1,s[1])
        east = (s[0]+1,s[1])

        for i in [north, south, west, east]:    
            if visited[i[0]][i[1]] == False:
                queue.append((i,h+1))
                visited[i[0]][i[1]] = True

                if(foodGrid[i[0]][i[1]]):    
                    return h
                
    return 0

" Verifica se existem fantasmas comiveis e retorna a distancia minima para eles."
"         - Se nenhum fantasma for comivel, retorna 0 "
def getMinDistEdibleGhost(agentPos,state):
    ghostPos = []
    ghostStates = state.getGhostStates()
    for gs in ghostStates:
        if gs.scaredTimer > 0:
            ghostPos.append(gs.getPosition())
            
    return minDistBFS(agentPos, ghostPos, state.getWalls())

def normalizeDistance(x):
    if x == 0:
        return 0
    return 1/(x**2)

" Realiza todo o calculo das features e retorna um array (1,5) com as features tratadas."
def calcFeatures(state, action):
    futureState = state.generatePacmanSuccessor(action)
    pos = state.getPacmanPosition()
    future_pos = futureState.getPacmanPosition()
    
    futureScore = futureState.getScore() - state.getScore()
    ghostDist = minDistBFS(future_pos, futureState.getGhostPositions(), state.getWalls())
    foodDist = findFoodMinDistBFS(future_pos,state.getFood(),state.getWalls())
    pillDist = minDistBFS(future_pos, state.getCapsules(), state.getWalls())
    edibleGhost = getMinDistEdibleGhost(future_pos,futureState)
    
    foodDist = normalizeDistance(foodDist)
    pillDist = normalizeDistance(pillDist)

    if ghostDist == edibleGhost:
        ghostDist = 0    # atribui zero para nao confundir fantasma comivel com perigoso
        
    else:
        if ghostDist == 1:
            futureScore -= 200 # reforco negativo
        else:
            futureScore += 200*foodDist # reforco positivo
        
        ghostDist = -normalizeDistance(ghostDist)

        
    edibleGhost = normalizeDistance(edibleGhost)    
    
    return np.array([[futureScore/500,ghostDist,foodDist,pillDist,edibleGhost]])

" Realiza todo o calculo das features e retorna um array (1,5) com as features tratadas."
" Necessario pois as vezes quando o pacman prede, o score final da morte nao eh previsto."
def calcFinalFeatures(state, action, lastScore, lastEatFood):
    pos = state.getPacmanPosition()
        
    futureScore = state.getScore() - lastScore
    ghostDist = minDistBFS(pos, state.getGhostPositions(), state.getWalls())
    foodDist = findFoodMinDistBFS(pos,state.getFood(),state.getWalls())
    pillDist = minDistBFS(pos, state.getCapsules(), state.getWalls())
    edibleGhost = 0

    if ghostDist != 0:
        ghostDist = -1/(ghostDist**2)
    else:
        ghostDist = 0
        
    if pillDist != 0:
        pillDist = 1/(pillDist**2)
    else:
        pillDist = 0
        
    if foodDist != 0:
        foodDist = 1/(foodDist**2)
    else:
        foodDist = 0
    if lastEatFood > 0:
        foodDist = lastEatFood
    
    return np.array([[futureScore/500,ghostDist,foodDist,pillDist,edibleGhost]])

### Agente
&emsp;&emsp;Calculamos os scores das possiveis ações do agente no método *getAction*, que determina a ação do pacman a cada iteração do jogo.

In [3]:
class RLAgent(Agent):
    "An agent that rocks until it can't."
    def __init__(self, W = None, Q=0, ls=0):
        self.W = W
        self.Q = Q
        self.lastScore = ls
        self.n_train = 0
        self.isTrain=True
        self.numActions = 0
        
    def setW(self, W):
        if self.isTrain:
            self.W = W
        
    def setQ(self, Q):
        self.Q = Q
        
    def setLastScore(self, lastScore):
        self.lastScore = lastScore
        
    def setNTrain(self, n_train):
        self.n_train = n_train
        
    def setIsTrain(self, isTrain):
        self.isTrain = isTrain
        
    def setNumActions(self, numActions):
        self.numActions = numActions
        
    def setLastEatFood(self, lastEatFood):
        self.lastEatFood = lastEatFood

    def getAction(self, state):
        "The agent receives a GameState (defined in pacman.py)."

        count = 0
        legalActions = state.getLegalPacmanActions()
        legalActions.remove(Directions.STOP) #remove a opcao Stop
        actionScore = []
        
        
        for action in legalActions:    
            stateVars = calcFeatures(state, action)
            actionValue = np.dot(self.W,stateVars.T)
            actionScore.append(actionValue[0][0])
            
        print("Action Score: ",actionScore)
        
        # --- Escolhe action ---
        #   - se estiver em modo de treino, utiliza logica do e-greedy, senao soh greedy
        if self.isTrain:
            epsilon = 0.4/self.n_train
            if epsilon < 0.1:
                epsilon = 0
            print("-- epsilon: ",epsilon)
            eGreedy = np.random.choice(2, p=[epsilon,1-epsilon])
            
            if eGreedy > 0:
                action = legalActions[actionScore.index(max(actionScore))]
            else:
                action = np.random.choice(legalActions)
        else:
            action = legalActions[actionScore.index(max(actionScore))]
        
        
        # --- Define variaveis ---
        gama = 0.9
        r = stateVars[0][0] # futureScore/500
        alfa = 0.008
        
        stateVars = calcFeatures(state, action)        
        chosenScore = np.dot(self.W,stateVars.T) # Q(s',a')
        
        self.setLastEatFood(stateVars[0][1])
        
        difference = (r+gama*chosenScore[0][0]) - self.Q # (r-gamma*Q(s',a')-Q(s,a))
        self.setQ(chosenScore[0][0])
        self.setLastScore(state.getScore())
        self.setW(self.W + alfa*difference*np.array(stateVars)) # atualiza pesos W
        
        print("W: ", self.W, "  -  ACTION: ",action)
        
        if action in state.getLegalPacmanActions():
            self.setNumActions(self.numActions+1)
            return action
        else:
            return Directions.STOP

### Approximate Q-Learning

&emsp;&emsp;O algoritmo funciona de forma simples. Primeiro identificamos quais features seriam utilizadas para o cálculo de Q, e também definimos um padrão para o cálculo de Reward (futureScore - presentScore). Assim, podemos definir o cálculo e atualização de Q da seguinte forma:\
\
$$ W=\{w_1,w_2,w_3,w_5,w_5\},  F=\{f_1,f_2,f_3,f_4,f_5\}$$\
$$ Q(s,a)=w_1*f_1+w_2*f_2+...+w_5*f_5 $$\
$$ W = W + \alpha*(r + \gamma*Q(s',a'))*F $$


&emsp;&emsp;Para a escolha de qual ação o agente deve escolher, utilizamos uma abordagem $\epsilon$-Greedy, com $\epsilon=0.3$ no início do treinamento, e diminuindo esse valor nos treinamentos seguintes, até chegar a zero.
<br>


In [4]:
def approxQLearning(pacman, layoutType, maxTrain=30, n_train=1):
    numGames=1
    gameLayout = layout.getLayout( layoutType )
    ghostType = loadAgent('RandomGhost', True)
    ghosts = [ghostType( i+1 ) for i in range( 4 )]

#     import textDisplay
#     textDisplay.SLEEP_TIME = 0.1
#     gameDisplay = textDisplay.PacmanGraphics()
    import graphicsDisplay
    gameDisplay = graphicsDisplay.PacmanGraphics(1.0, frameTime = 0.1)

    results = []
    gamesScores = []
    numActions = []
    
    count = 0
    while count < maxTrain:
        print("------------------------------------------ train --------------------")
        pacman.setNumActions(0)
        count += 1
        pacman.setNTrain(n_train+1)
        games = runGames( gameLayout, pacman, ghosts, gameDisplay, numGames, False)
        scores = [game.state.getScore() for game in games]
        
        # se o pacman perdeu, recalcula o ultimo Q(s,a), para pegar o reforco negativo da morte
        if games[0].state.isLose():
            stateVars = calcFinalFeatures(games[0].state, None, pacman.lastScore, pacman.lastEatFood)
            actionValue = np.dot(pacman.W,stateVars.T)

            gama = 0.9
            r = stateVars[0][0]
            alfa = 0.008

            difference = (r+gama*actionValue[0][0]) - pacman.Q
            pacman.setW(pacman.W + alfa*difference*stateVars)
        
        results.append(games[0].state.isWin())
        gamesScores.append(scores[0])
        numActions.append(pacman.numActions)

    print(results)
    print("Wins: ",results.count(True)," Loss: ",results.count(False))
    print(gamesScores)
    return gamesScores, results, numActions

### Treinamento
&emsp;&emsp;Para o treinamento do modelo de aprendizagem por reforco, treinamos para os 3 mapas da seguinte maneira:
- smallClassic: 15 jogos
- mediumClassic: 10 jogos
- originalClassic: 10 jogos

&emsp;&emsp;Escolhemos esse numero de treinamento atraves de observacoes. Percebemos que quando o pacman treinava muitos jogos em um mapa, os pesos convergiam para um overfit. \
&emsp;&emsp;Para os valores de $\alpha$ e $\gamma$, utilizamos os valores a seguir:
- $\alpha=0.008$
- $\gamma=0.9$

In [5]:
def printGraphs(gamesScores,results,numActions):
    fig = plt.figure(figsize=(18, 3))
    plt.subplot(131)
    plt.xlabel('Treino N')
    plt.ylabel('Scores')
    plt.title('Game Scores')
    plt.plot(range(1,len(gamesScores)+1),gamesScores[:])

    plt.subplot(132)
    plt.xlabel('Treino N')
    plt.ylabel('Vitórias')
    plt.title('Vitórias')
    plt.plot(range(1,len(gamesScores)+1),results[:])

    plt.subplot(133)
    plt.xlabel('Treino N')
    plt.ylabel('Num Actions')
    plt.title('Number of Actions')
    plt.plot(range(1,len(gamesScores)+1),numActions[:])
    plt.show()

    print("Wins: ",results.count(True)," Loss: ",results.count(False))
    print("Avg Score: ",sum(gamesScores)/len(gamesScores))
    print("Avg NumActions: ",sum(numActions)/len(numActions))

In [6]:
W = np.zeros((1,5))
W[:] = 0.7
pacman = RLAgent(W)

In [ ]:
layoutType='smallClassic'

gamesScores, results, numActions = approxQLearning(pacman, layoutType, 15)

layout:  smallClassic
------------------------------------------ train --------------------
Action Score:  [0.9977523760330578, 0.9998857142857142]
-- epsilon:  0.2
W:  [[0.70440705 0.69989457 0.71054318 0.70021517 0.7       ]]   -  ACTION:  West
Action Score:  [0.994839098565273, 1.0134998918996294]
-- epsilon:  0.2
W:  [[0.70551145 0.69985329 0.71318529 0.70028856 0.7       ]]   -  ACTION:  West
Action Score:  [0.2507928405707436, 1.0166602506972435]
-- epsilon:  0.2
W:  [[0.70657984 0.69978229 0.71574125 0.7003908  0.7       ]]   -  ACTION:  West
Action Score:  [0.9868105256830622, 0.24819703877677998, 1.0354276466404693]
-- epsilon:  0.2
W:  [[0.70628562 0.69986568 0.71499068 0.7003074  0.7       ]]   -  ACTION:  East
Action Score:  [0.24797267516634044, 0.9961100244756385]
-- epsilon:  0.2
W:  [[0.70961704 0.69953087 0.7233611  0.70064222 0.7       ]]   -  ACTION:  West
Action Score:  [1.0114620690312615, 0.2555688878728033, 1.049495018325164]
-- epsilon:  0.2
W:  [[0.70934414 0.6

Action Score:  [0.2899521551468099, 1.1229631563374582]
-- epsilon:  0.2
W:  [[0.76759633 0.71252916 0.80015119 0.70317656 0.7       ]]   -  ACTION:  West
Action Score:  [1.1208155898303211, 0.27488813468878565]
-- epsilon:  0.2
W:  [[0.76754206 0.71253181 0.80002136 0.70317392 0.7       ]]   -  ACTION:  South
Action Score:  [0.2662557981343038, 1.1172995680953526]
-- epsilon:  0.2
W:  [[0.76855447 0.71248238 0.80244339 0.70321176 0.7       ]]   -  ACTION:  South
Action Score:  [0.2757399896521675, 1.1235099641477597]
-- epsilon:  0.2
W:  [[0.76959733 0.71243147 0.80493827 0.70326268 0.7       ]]   -  ACTION:  South
Action Score:  [0.27310416652168246, 1.1176677659292433]
-- epsilon:  0.2
W:  [[0.77060184 0.71233534 0.80734139 0.70332943 0.7       ]]   -  ACTION:  South
Action Score:  [0.2771705345646714, 1.1290927261476151]
-- epsilon:  0.2
W:  [[0.77166027 0.71223406 0.80987352 0.70343071 0.7       ]]   -  ACTION:  East
Action Score:  [1.1318773023075652, 0.27784654876013937]
-- epsi

Action Score:  [0.28501181583692714, 1.1391445890202203]
-- epsilon:  0.2
W:  [[0.77318473 0.7138587  0.7976747  0.70683657 0.69907102]]   -  ACTION:  West
Action Score:  [0.2916524234192672, 1.160819190674579]
-- epsilon:  0.2
W:  [[0.77426682 0.71384338 0.80026345 0.70699837 0.69907102]]   -  ACTION:  West
Action Score:  [0.14688878333693817, 1.1827530901995762]
-- epsilon:  0.2
W:  [[0.7752273  0.7138291  0.80267671 0.70726651 0.69907102]]   -  ACTION:  West
Action Score:  [0.44923424128091594, 0.3176730375172986, 1.1667020433764943]
-- epsilon:  0.2
W:  [[0.77618127 0.71381559 0.80495894 0.70740915 0.69907102]]   -  ACTION:  West
Action Score:  [0.35000721539772295, 0.3516827569146821, 1.151799781188044]
-- epsilon:  0.2
W:  [[0.77714407 0.71379656 0.80726229 0.70750128 0.69907102]]   -  ACTION:  West
Action Score:  [0.3133822178135831, 0.10664789172751786]
-- epsilon:  0.2
W:  [[0.77695403 0.71386484 0.80674588 0.70698488 0.69907102]]   -  ACTION:  East
Action Score:  [0.195269978

Action Score:  [1.23315715608003, 1.2544393685694801]
-- epsilon:  0.2
W:  [[0.84770137 0.76445545 0.89525612 0.7118162  0.69907102]]   -  ACTION:  West
Action Score:  [0.30947100957296414, 1.256833068933394]
-- epsilon:  0.2
W:  [[0.84868688 0.76438996 0.8976138  0.71191051 0.69907102]]   -  ACTION:  West
Action Score:  [1.224365840832547, 0.3061170031429892, 1.2756262739296653]
-- epsilon:  0.2
W:  [[0.84972095 0.76432124 0.90008765 0.71206513 0.69907102]]   -  ACTION:  West
Action Score:  [1.2455280464962826, 1.3224468341142561]
-- epsilon:  0.2
W:  [[0.85083308 0.76427967 0.90274826 0.71236075 0.69907102]]   -  ACTION:  West
Action Score:  [1.4288438758686077, 0.341649383240418, 1.2952762351458245]
-- epsilon:  0.2
W:  [[0.85210886 0.76424915 0.90580036 0.71312378 0.69907102]]   -  ACTION:  North
Action Score:  [1.9674631458474336, 1.3165331678700272]
-- epsilon:  0.2
W:  [[0.854583   0.76418996 0.91171934 0.71904276 0.69907102]]   -  ACTION:  North
Action Score:  [1.44105382392542

Action Score:  [1.341133031277687, 1.3954160818270158, 0.2747337711475056]
-- epsilon:  0.2
W:  [[0.88128783 0.76335659 0.97761041 0.7193352  0.69907102]]   -  ACTION:  East
Action Score:  [1.44100512084524, 1.3604129068749795, 1.2885040148922589]
-- epsilon:  0.2
W:  [[0.88228931 0.76309038 0.98000632 0.71993417 0.69907102]]   -  ACTION:  South
Action Score:  [1.3263623312926145, 1.983949605695855, -0.04012589482169737]
-- epsilon:  0.2
W:  [[0.88216408 0.76312367 0.97970671 0.71963457 0.69907102]]   -  ACTION:  South
Action Score:  [0.041861726555747936, 1.4435684235600248, 1.040471478882184]
-- epsilon:  0.2
W:  [[0.87986763 0.7637341  0.97421281 0.71826109 0.69907102]]   -  ACTION:  South
Action Score:  [-0.15562659361890852, 1.3369449221426053]
-- epsilon:  0.2
W:  [[0.8804618  0.76357616 0.97563427 0.71841903 0.69907102]]   -  ACTION:  South
Action Score:  [0.4249567850755989, 1.3580254403381842, 1.3580254403381842]
-- epsilon:  0.2
W:  [[0.88147596 0.76347912 0.97806049 0.718570

...

Action Score:  [0.18089173505444567, 1.2919038369974079]
-- epsilon:  0.2
W:  [[1.11250935 0.88229036 1.24865374 0.76177975 0.7077666 ]]   -  ACTION:  West
Action Score:  [0.3885791501912531, 0.338249081032197, 1.2969895754077587]
-- epsilon:  0.2
W:  [[1.11249399 0.88229036 1.24780084 0.76177975 0.70773248]]   -  ACTION:  West
Action Score:  [0.35395850147901825, 1.2874849650034799]
-- epsilon:  0.2
W:  [[1.11247668 0.88229036 1.24683881 0.76177975 0.70770576]]   -  ACTION:  West
Action Score:  [0.329143243104728, 1.2779212953350931]
-- epsilon:  0.2
W:  [[1.11245949 0.88229036 1.24588397 0.76177975 0.70769084]]   -  ACTION:  West
Action Score:  [0.05204104783235963, 0.08670049827185783]
-- epsilon:  0.2
W:  [[1.11247872 0.88229036 1.24528302 0.76177975 0.7075406 ]]   -  ACTION:  West
Action Score:  [0.05531756139102817, 0.18053260355191406]
-- epsilon:  0.2
W:  [[1.11251886 0.88227559 1.24538811 0.76177975 0.70755228]]   -  ACTION:  West
Action Score:  [0.09335687788796272, 0.4153235

Action Score:  [0.40938998942462146, 1.6880377278370258, 0.41190999556723695]
-- epsilon:  0.2
W:  [[1.13586265 0.88374409 1.29451896 0.76843233 0.70765413]]   -  ACTION:  East
Action Score:  [-0.0017793210727561348, 0.34875724824303145]
-- epsilon:  0.2
W:  [[1.13486215 0.88487846 1.29196665 0.76827281 0.70765413]]   -  ACTION:  West
Action Score:  [0.16581015497490792, 1.6610009958172556, 0.1683295681359136]
-- epsilon:  0.2
W:  [[1.13864661 0.88382193 1.30147536 0.76846687 0.70765413]]   -  ACTION:  East
Action Score:  [1.7005735059829619, 0.4136106247930673]
-- epsilon:  0.2
W:  [[1.13853798 0.88385081 1.30121548 0.76845965 0.70765413]]   -  ACTION:  East
Action Score:  [1.709657095913589, 0.41720940959624997]
-- epsilon:  0.2
W:  [[1.13832436 0.88390759 1.30070442 0.76843921 0.70765413]]   -  ACTION:  East
Action Score:  [1.769307230351535, 1.5768926398733119, 0.40283320177437193]
-- epsilon:  0.2
W:  [[1.13825989 0.88391723 1.30055018 0.76842957 0.70765413]]   -  ACTION:  North
A

Action Score:  [0.5494364514664722, -0.958362258919206, 1.7861582319513754]
-- epsilon:  0.2
W:  [[1.15943898 0.87900064 1.35246993 0.78388877 0.70765413]]   -  ACTION:  West
Action Score:  [0.44117451639076194, 1.6487208130836961]
-- epsilon:  0.2
W:  [[1.15867764 0.87986384 1.35052773 0.78302557 0.70765413]]   -  ACTION:  East
Action Score:  [0.3400029663111237, 0.15041451381359283, 1.6406545634082175]
-- epsilon:  0.2
W:  [[1.16324163 0.87699701 1.36199504 0.78374227 0.70765413]]   -  ACTION:  West
Action Score:  [0.3223296624678341, -1.0912522109148635]
-- epsilon:  0.2
W:  [[1.16109931 0.88246213 1.35652993 0.78131333 0.70765413]]   -  ACTION:  East
Action Score:  [0.17472056397037827, 0.028224314045515286, -1.1049613424497362]
-- epsilon:  0.2
W:  [[1.16056719 0.88559629 1.35513697 0.77817917 0.70765413]]   -  ACTION:  North
Action Score:  [0.756610502174659, -2.0874629010393666]
-- epsilon:  0.2
W:  [[1.16026303 0.88738783 1.35434072 0.771013   0.70765413]]   -  ACTION:  North
A

Action Score:  [0.28076150500649777, 1.9025592589072153]
-- epsilon:  0.2
W:  [[1.20913469 0.88625671 1.42982957 0.77510751 0.70833418]]   -  ACTION:  West
Action Score:  [1.9013875863360465, 0.4997785687779043, 0.47286511341076976]
-- epsilon:  0.2
W:  [[1.20882266 0.88630337 1.42908308 0.77508678 0.70833418]]   -  ACTION:  North
Action Score:  [1.909980465224355, 0.4821193244852942]
-- epsilon:  0.2
W:  [[1.20854041 0.88634557 1.42840784 0.77505977 0.70833418]]   -  ACTION:  North
Action Score:  [1.9573982576681157, 0.4724476481480864, 1.8997105722320664]
-- epsilon:  0.2
W:  [[1.20944221 0.88628564 1.43056526 0.77519461 0.70833418]]   -  ACTION:  North
Action Score:  [1.9532716810007527, 1.91831200424495, 0.5484511741462507]
-- epsilon:  0.2
W:  [[1.20910295 0.88629832 1.42975363 0.77516214 0.70833418]]   -  ACTION:  North
Action Score:  [0.510529720585908, 0.15675362075694502]
-- epsilon:  0.2
W:  [[1.20794985 0.88648217 1.42681204 0.77442675 0.70833418]]   -  ACTION:  South
Action

Action Score:  [2.1712560235580005, 0.6712753411270691, 1.13927670199065]
-- epsilon:  0.2
W:  [[1.25092355 0.9239778  1.46878072 0.76270482 0.70833418]]   -  ACTION:  North
Action Score:  [2.06500460991408, 1.2410833703655775, 2.057555053431594]
-- epsilon:  0.2
W:  [[1.2512755  0.92396741 1.46962271 0.76279837 0.70833418]]   -  ACTION:  North
Action Score:  [2.1469229412032687, 2.032694667560735, 2.0289237568063556]
-- epsilon:  0.2
W:  [[1.25188551 0.92395535 1.47108207 0.76288958 0.70833418]]   -  ACTION:  East
Action Score:  [2.018469441126298, 2.0448584597067767]
-- epsilon:  0.2
W:  [[1.25254039 0.92393889 1.47272749 0.7630724  0.70833418]]   -  ACTION:  West
Action Score:  [2.147570125549302, 2.009691205905703, 2.0295448574250283]
-- epsilon:  0.2
W:  [[1.25351455 0.92390065 1.47517513 0.76368431 0.70833418]]   -  ACTION:  South
Action Score:  [2.0444917873387225, 1.2408865752992329, 2.058334119054808]
-- epsilon:  0.2
W:  [[1.25392563 0.92387333 1.47615858 0.76379359 0.7083341

In [ ]:
printGraphs(gamesScores,results,numActions)

In [ ]:
layoutType='mediumClassic'

gamesScores, results, numActions = approxQLearning(pacman, layoutType,10, pacman.n_train)

In [ ]:
printGraphs(gamesScores,results,numActions)

In [ ]:
layoutType='originalClassic'

gamesScores, results, numActions = approxQLearning(pacman, layoutType,10,pacman.n_train)

In [ ]:
printGraphs(gamesScores,results,numActions)

#### Análise para o resultado do treinamento
&emsp;&emsp;Percebemos através do treinamento que o pacman aprende o equilíbrio entre procurar por comidas e fugir dos fantasmas, e com as features de pílula e fantasmas comíveis, o agente aprende que essas outras interações do jogo são positivas e têm boa recompensa.\
&emsp;&emsp;Alguns desafios foram encontrados no começo do desenvolvimento, observamos que se em um futuro estado havia uma comida e um fantasma, o peso de comer a comida era maior que o de fugir do fantasma, e que o reforço negativo com a morte do agente, na feature *futureScore*, não era suficiente para equilibrar esses pesos. Assim, adicionamos um reforço negativo em *futureScore* caso houvesse um fantasma naquele estado, pois a função *getScore* do GameState não calculava a morte do pacman. Assim, o agente aprendeu que fugir do fantasma era mais importante que comer uma comida.\
&emsp;No entanto, com essa modificação o agente ficou enviesado para fugir dos fantasmas ao invés de buscar as comidas para ganhar o jogo, e o treinamento não era capaz de balancear esses pesos. Então, adicionamos um reforço positivo também em *futureScore*, para que, caso não ohouvesse um fantasma no próximo estado, o agente fosse em busca das comidas disponíveis.

### Teste 
&emsp;&emsp;Teste do modelo final do agente, resultante do treinamento anterior. Realizamos 30 jogos para cada um dos mapas.

In [ ]:
layoutType='smallClassic'
pacman.setIsTrain(False)
gamesScores, results, numActions = approxQLearning(pacman, layoutType,30,pacman.n_train)

In [ ]:
printGraphs(gamesScores,results,numActions)

In [ ]:
layoutType='mediumClassic'
pacman.setIsTrain(False)
gamesScores, results, numActions = approxQLearning(pacman, layoutType,30,pacman.n_train)

In [ ]:
printGraphs(gamesScores,results,numActions)

In [ ]:
layoutType='originalClassic'
pacman.setIsTrain(False)
gamesScores, results, numActions = approxQLearning(pacman, layoutType,30,pacman.n_train)

In [ ]:
printGraphs(gamesScores,results,numActions)

#### Análise dos resultados dos testes
&emsp;&emsp;Percebemos que as considerações do passo de treinamento quanto às features e aos reforços foram bem sucedidos, como podemos ver no desempenho do modelo nos casos de teste. \
&emsp;&emsp;Fomos capazes de resolver alguns desafios do jogo, mas não todos. Percebemos que quase todas as vezes que o pacman morre, é por que ele foi encurralado por dois fantasmas, um cenário que não é previsto no nosso modelo.\
&emsp;&emsp;Como consideração final, esta seria uma melhoria a ser implementada.